# Performance evaluation

## Tasks generation

In [1]:
# import random number generator
import random

In [2]:
task_set_size_min = 2
task_set_size_max = 30
sets_per_size = 1

task_set_weight_lower_bound = 50
task_set_weight_upper_bound = 100

task_set_size_vector = [i for i in range(task_set_size_min, task_set_size_max + 1)]

task_set_weight_matrix = []
task_set_period_matrix = []
task_set_computing_time_matrix = []
for task_set_size in task_set_size_vector:
    j = task_set_size - task_set_size_vector[0]
    task_set_weight_matrix.append([])
    task_set_period_matrix.append([])
    task_set_computing_time_matrix.append([])
    for n in range(sets_per_size):
        task_set_weight_matrix[j].append([])
        task_set_period_matrix[j].append([])
        task_set_computing_time_matrix[j].append([])
        for i in range(task_set_size):
            task_set_weight_matrix[j][n].append(random.randint(1, task_set_weight_upper_bound))
            # task_set_weight_matrix[j][n].append(random.randint(
            #     max(1, int(task_set_weight_lower_bound / task_set_size)),
            #     max(1, int(task_set_weight_upper_bound / task_set_size))
            #     ))
            task_set_period_matrix[j][n].append(random.randint(10, 600))
            task_set_computing_time_matrix[j][n].append(random.randint(1, task_set_period_matrix[j][n][i]))
        while sum(task_set_weight_matrix[j][n]) < task_set_weight_lower_bound:
            _pos = random.randint(0, task_set_size - 1)
            if task_set_weight_matrix[j][n][_pos] < task_set_weight_upper_bound:
                task_set_weight_matrix[j][n][_pos] += 1
        while sum(task_set_weight_matrix[j][n]) > task_set_weight_upper_bound:
            _pos = random.randint(0, task_set_size - 1)
            if task_set_weight_matrix[j][n][_pos] > 1:
                task_set_weight_matrix[j][n][_pos] -= 1
            
sum_task_set_weights = []
for task_set_size in task_set_size_vector:
    j = task_set_size - task_set_size_vector[0]
    sum_task_set_weights.append([])
    for n in range(sets_per_size):
        sum_task_set_weights[j].append(sum(task_set_weight_matrix[j][n]))


Select one task set and write header file with its parameters.

In [16]:
selected_task_set_size = 7
selected_task_set_from_size = 1

j = selected_task_set_size - task_set_size_vector[0]
n = selected_task_set_from_size - 1

f = open("../tasks.h", "w+")
f.write(
"/**\n"
" * @file tasks.h\n"
" * @author Vinicius Pimenta Bernardo (viniciuspibi@gmail.com)\n"
" * @brief This file contains the tasks definitions.\n"
" * @version 0.1\n"
" * @date 2022-02-01\n"
" * \n"
" * @copyright Copyright (c) 2022\n"
" * \n"
" */\n"
"\n"
"#ifndef TASKS_H_\n"
"#define TASKS_H_\n"
"\n"
"#include <stdint.h>\n"
"#include <stdbool.h>\n"
"\n"
"#define TASK_COUNT " + str(selected_task_set_size) + "\n"
"#define SUM_OF_WEIGHTS " + str(sum_task_set_weights[j][n]) + "\n"
"\n"
"const uint32_t      weights[TASK_COUNT] = {"
)
for i in range(selected_task_set_size):
    if i == selected_task_set_size - 1:
        f.write(str(task_set_weight_matrix[j][n][i]))
    else:
        f.write(str(task_set_weight_matrix[j][n][i]) + ", ")
f.write(
"};\n"
"const uint32_t      task_period[TASK_COUNT] = {"
)
for i in range(selected_task_set_size):
    if i == selected_task_set_size - 1:
        f.write(str(task_set_period_matrix[j][n][i]))
    else:
        f.write(str(task_set_period_matrix[j][n][i]) + ", ")
f.write(
"};\n"
"const uint32_t      task_computing_time[TASK_COUNT] = {"
)
for i in range(selected_task_set_size):
    if i == selected_task_set_size - 1:
        f.write(str(task_set_computing_time_matrix[j][n][i]))
    else:
        f.write(str(task_set_computing_time_matrix[j][n][i]) + ", ")
f.write(
"};\n"
"\n"
"volatile uint32_t   values[TASK_COUNT];\n"
"volatile bool       task_on_off_status[TASK_COUNT];\n"
"volatile uint32_t   task_deadline[TASK_COUNT];\n"
"volatile uint32_t   task_executed_time[TASK_COUNT];\n"
"volatile bool       task_already_executed[TASK_COUNT];\n"
"\n"
"#endif /* TASKS_H_ */\n"
)
f.close()


In [ ]:
# Print the tasks sets
for task_set_size in task_set_size_vector:
    j = task_set_size - task_set_size_vector[0]
    for n in range(sets_per_size):
        print(task_set_weight_matrix[j][n])
        print(task_set_period_matrix[j][n])
        print(task_set_computing_time_matrix[j][n])
        print(sum(task_set_weight_matrix[j][n]))

## HIL time measurement

In [4]:
# Import python serial
import serial

Find maximum number of tasks by trial and error.

In [18]:
s = serial.Serial('COM4', 9600, timeout=5)

txData = [50, 30]
for i in range(10):
    rxData = []
    for i in range(len(txData)):
        s.write(bytes([txData[i]]))
    for i in range(selected_task_set_size):
        rxData.append(s.read())
    print(rxData)

s.close()

Magical number is 7.